# 年龄预测

In [1]:
# coding=utf-8
"""
Credit by Liao Jialing

"""
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# 1. 读入清洗整理后的用户基本信息宽表

In [2]:
dataset = pd.read_csv('E:/user_all_cleared.csv')

D:\Users\Jialing Liao\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 将target放在第一列

In [3]:
age_label = dataset.pop('member_age')

In [4]:
dataset.insert(0,'member_age',age_label)

In [5]:
dataset.shape

(503862, 152)

In [6]:
dataset.head()

,member_age,customerid,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now,...,age_from_song_language_3.0,age_from_song_language_not_known,shop_com_regioncode_310000.0,shop_com_regioncode_350000.0,shop_com_regioncode_440000.0,shop_com_regioncode_500000.0,shop_com_regioncode_not_known,shop_com_managetype_1.0,shop_com_managetype_2.0,shop_com_managetype_not_known
0,0,5792237,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
1,0,5792238,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
2,0,5792239,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
3,0,5792241,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
4,0,5792242,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


# 2. 分出测试集和训练集

In [7]:
train = dataset[dataset['member_age']!=0]
test = dataset[dataset['member_age']==0]
test_id = test.customerid
test = test.drop('customerid',axis=1)
X = train.drop(['member_age','customerid'],axis=1)
y = train.member_age.values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1026)

In [8]:
train.shape

(15559, 152)

In [9]:
test.shape

(488303, 151)

In [10]:
print(X_train.shape,X_test.shape,test.shape)

((12447, 150), (3112, 150), (488303, 151))


# 3. 进行特征选择

In [17]:
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## 特征选择

In [18]:
important_features = zip(X_train.columns.values, model.feature_importances_)
important_features = sorted(important_features, key=lambda x: x[1], reverse=True)

In [19]:
for r in important_features:
    print r[0], r[1]

lastconsume_diff_now 0.27097548258
consumetotal 0.184207005144
integralbalance 0.156050399767
consumetimes 0.0711753648662
accountbalance 0.0707379256897
lastconsume_diff_create_hour 0.070507736056
accountcash 0.0690803300494
gender 0.0539815873258
lastconsume_diff_create_day 0.0527581429113
age_from_song_language_2.0 3.977564225e-05
song_play_cnt 3.93727355378e-05
age_from_song_language_not_known 3.54553656357e-05
des_charge_max 1.48446033596e-05
des_realcharge_max 1.42146856091e-05
des_charge_mean 1.36467571823e-05
des_guestnumber_mean 1.33658883391e-05
des_use_reg 1.33208750728e-05
des_work_realchargesum 1.28190283622e-05
des_workday_cnt 1.1994643713e-05
des_companyid 1.13784169595e-05
des_realcharge_sum 1.06065891706e-05
des_guestnumber_min 1.04717697387e-05
use_time_sum 1.01672815783e-05
des_use_night_realchargesum 8.98097085629e-06
des_realcharge_mean 8.86206849297e-06
des_guestnumber_max 8.79922022662e-06
des_charge_sum 8.61148350985e-06
get_com_roomnum 8.57910503077e-06
des_mor

## 使用1e-6特征选出特征

In [20]:
THRESHOLD = 1e-6
importance_features = pd.DataFrame(important_features, columns=['feature', 'score'])
filtered_features = importance_features.loc[importance_features['score']>THRESHOLD, 'feature'].values

In [21]:
filtered_features

array(['lastconsume_diff_now', 'consumetotal', 'integralbalance',
       'consumetimes', 'accountbalance', 'lastconsume_diff_create_hour',
       'accountcash', 'gender', 'lastconsume_diff_create_day',
       'age_from_song_language_2.0', 'song_play_cnt',
       'age_from_song_language_not_known', 'des_charge_max',
       'des_realcharge_max', 'des_charge_mean', 'des_guestnumber_mean',
       'des_use_reg', 'des_work_realchargesum', 'des_workday_cnt',
       'des_companyid', 'des_realcharge_sum', 'des_guestnumber_min',
       'use_time_sum', 'des_use_night_realchargesum',
       'des_realcharge_mean', 'des_guestnumber_max', 'des_charge_sum',
       'get_com_roomnum', 'des_morning_cnt', 'des_use_night_cnt',
       'get_coupon_crosscom', 'des_realcharge_min', 'use_company_id',
       'des_charge_min', 'get_coupon_type_cnt', 'des_company_cnt',
       'des_procharge_sum', 'use_coupon_type_cnt', 'shop_cnt',
       'des_afternoon_cnt', 'use_coupon_sum', 'des_night_cnt', 'des_cnt',
       'sh

In [22]:
len(filtered_features)

72

## 按选择到的特征分别处理训练集和测试集

In [27]:
X_test = X_test[filtered_features]

In [28]:
X_train = X_train[filtered_features]

In [29]:
X_train.head()

,lastconsume_diff_now,consumetotal,integralbalance,consumetimes,accountbalance,lastconsume_diff_create_hour,accountcash,gender,lastconsume_diff_create_day,age_from_song_language_2.0,...,shop_weekday_cnt,shop_real_sum,shop_afternoon_sum,get_com_managetype_1.0,shop_weekday_sum,shop_afternoon_cnt,shop_original_sum_max,des_use_afternoon_cnt,shop_real_sum_avg,shop_night_cnt
270032,380,597,547,2,53,0,53,1,0,0,...,0,0,0,0,0,0,0,0,0,0
272418,369,1256,124,3,0,23,0,1,4,0,...,0,0,0,0,0,0,0,0,0,0
277465,359,544,0,2,56,22,56,1,1,0,...,0,0,0,0,0,0,0,0,0,0
272332,374,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
279727,356,420,420,1,0,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
test = test[filtered_features]

# 3. Xgboost模型训练

In [50]:
params={
'booster':'gbtree',
'objective': 'reg:linear', # 这里用户年龄是连续的，因此采用了默认的线性分类器
'gamma':0.05,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 [0:]
'max_depth':12, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'subsample':0.4, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
'colsample_bytree':0.7, # 构建树树时的采样比率 (0:1]
#'min_child_weight':12, # 节点的最少特征数
'silent':1 ,
'eta': 0.01, # 如同学习率
'seed':710,
'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}

In [51]:
plst = list(params.items())

In [33]:
num_rounds = 600 # 迭代次数
xgtest = xgb.DMatrix(test)
xgtrain = xgb.DMatrix(X_train, label=y_train)
xgval = xgb.DMatrix(X_test, label=y_test)

### return训练和验证的标准误差

In [34]:
watchlist = [(xgtrain, 'train'),(xgval, 'val')]

### 训练模型

In [35]:
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50)

Will train until val error hasn't decreased in 50 rounds.
[0]	train-rmse:26.754517	val-rmse:27.076355
[1]	train-rmse:26.507433	val-rmse:26.829119
[2]	train-rmse:26.263714	val-rmse:26.585361
[3]	train-rmse:26.023752	val-rmse:26.345247
[4]	train-rmse:25.785013	val-rmse:26.106462
[5]	train-rmse:25.549324	val-rmse:25.870890
[6]	train-rmse:25.315861	val-rmse:25.637770
[7]	train-rmse:25.084364	val-rmse:25.406252
[8]	train-rmse:24.855721	val-rmse:25.177628
[9]	train-rmse:24.629303	val-rmse:24.951685
[10]	train-rmse:24.405464	val-rmse:24.727772
[11]	train-rmse:24.184996	val-rmse:24.507671
[12]	train-rmse:23.966570	val-rmse:24.289457
[13]	train-rmse:23.749325	val-rmse:24.072374
[14]	train-rmse:23.535444	val-rmse:23.858900
[15]	train-rmse:23.325338	val-rmse:23.648846
[16]	train-rmse:23.116831	val-rmse:23.440639
[17]	train-rmse:22.910149	val-rmse:23.233952
[18]	train-rmse:22.706268	val-rmse:23.030333
[19]	train-rmse:22.503792	val-rmse:22.828291
[20]	train-rmse:22.304035	val-rmse:22.628567
[21]	tr

In [36]:
model.save_model('E:/age.model')

In [37]:
preds = model.predict(xgtest,ntree_limit=model.best_iteration)

In [38]:
preds

array([ 25.05296516,  25.05296516,  25.05296516, ...,  25.05296516,
        25.05296516,  25.05296516], dtype=float32)

# 4. 对预测结果进行处理和保存

In [107]:
# np.savetxt('E:/age.csv',np.c_[range(1,len(test)+1),preds],
#                 delimiter=',',header='id,Label',comments='',fmt='%d')

In [39]:
df = pd.DataFrame(preds)

In [40]:
df.columns = ['member_age']

In [41]:
df['member_age'] = map(int,df['member_age'])

In [42]:
pd.value_counts(df['member_age'])

25    479161
23      5671
26      2412
24      1059
Name: member_age, dtype: int64

In [43]:
type(test_id)

pandas.core.series.Series

In [44]:
df_y = pd.DataFrame(test_id)

In [45]:
age_predicted = pd.concat([df_y,df],axis=1)

In [46]:
age_predicted = age_predicted[age_predicted['member_age'].notnull()]

## 最终预测了488303项

In [47]:
age_predicted.shape

(488303, 2)

In [48]:
age_predicted.head()

,customerid,member_age
0,5792237,25
1,5792238,25
2,5792239,25
3,5792241,25
4,5792242,25


## 保存预测结果到csv

In [49]:
age_predicted.to_csv('E:/age_predicted.csv',index=False)